In [1]:
import sqlite3
import time
import argparse
import pandas as pd
import datetime
import numpy as np

In [17]:
with sqlite3.connect("timelogger.db") as con:
    cursor = con.cursor()
    now = datetime.datetime.now()
    now = now - datetime.timedelta(days=1)
    t = cursor.execute("""INSERT INTO timelog (id, start_timestamp, end_timestamp)
                    VALUES (?, ?, ?)""",
                    (1000001, now.timestamp(), 0))

In [33]:
with sqlite3.connect("timelogger.db") as con:
    cursor = con.cursor()
    cursor.execute("""select * from timelog """)
    for log in cursor.fetchall():
        print(f"start = {log[1]}, end = {log[2]}")

start = 1720382372.06768, end = 1720382384.551723


In [20]:
datetime.datetime.fromtimestamp(1720299600) + datetime.timedelta(microseconds=1)

datetime.datetime(2024, 7, 7, 0, 0, 0, 1)

In [21]:
with sqlite3.connect("timelogger.db") as con:
    data = pd.read_sql_query("SELECT * FROM timelog", con, index_col="id")

In [34]:
datetime.datetime.fromtimestamp(data.iloc[0]["end_timestamp"]).day

7

In [29]:
def get_day(timestamp):
    date = datetime.datetime.fromtimestamp(timestamp)
    delta = datetime.timedelta(days=0, hours=date.hour, minutes=date.minute, seconds=date.second, microseconds=date.microsecond)
    return date - delta

def get_month(timestamp):
    date = datetime.datetime.fromtimestamp(timestamp)
    res = datetime.datetime(year=date.year, month=date.month)
    return res

In [34]:
data["work_time"] = np.array(list(map(datetime.datetime.fromtimestamp ,data["end_timestamp"]))) -\
                    np.array(list(map(datetime.datetime.fromtimestamp ,data["start_timestamp"] )))
data["day"] = np.array(list(map(get_day ,data["start_timestamp"] )))
data["month"] = np.array(list(map(get_month ,data["start_timestamp"] )))
data[["month", "work_time"]].groupby("month").sum()

,work_time
month,
2024-06-30,1 days 01:44:10.580604


In [7]:
?pd.read_sql_query

Signature:
pd.read_sql_query(
    sql,
    con,
    index_col: 'str | list[str] | None' = None,
    coerce_float: 'bool' = True,
    params: 'list[str] | dict[str, str] | None' = None,
    parse_dates: 'list[str] | dict[str, str] | None' = None,
    chunksize: 'int | None' = None,
    dtype: 'DtypeArg | None' = None,
    dtype_backend: 'DtypeBackend | lib.NoDefault' = <no_default>,
) -> 'DataFrame | Iterator[DataFrame]'
Docstring:
Read SQL query into a DataFrame.

Returns a DataFrame corresponding to the result set of the query
string. Optionally provide an `index_col` parameter to use one of the
columns as the index, otherwise default integer index will be used.

Parameters
----------
sql : str SQL query or SQLAlchemy Selectable (select or text object)
    SQL query to be executed.
con : SQLAlchemy connectable, str, or sqlite3 connection
    Using SQLAlchemy makes it possible to use any DB supported by that
    library. If a DBAPI2 object, only sqlite3 is supported.
index_col : str or